In [1]:
# -----------------------------
# 1. Install & Import Libraries
# -----------------------------
!pip install yfinance pandas numpy matplotlib tensorflow scikit-learn --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# -----------------------------
# 2. Download Stock Data
# -----------------------------
# You can change 'AAPL' to any stock ticker (e.g., 'TSLA', 'MSFT')
df = yf.download('AAPL', start='2019-01-01', end='2024-12-31')

print("Data Shape:", df.shape)
df.head()

# -----------------------------
# 3. Plot Closing Price History
# -----------------------------
plt.figure(figsize=(10,6))
plt.plot(df['Close'], label='Close Price')
plt.title('Apple Stock Price History')
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.legend()
plt.show()

# -----------------------------
# 4. Prepare the Data
# -----------------------------
# Use only the 'Close' column
data = df[['Close']]

# Scale the data (0 to 1 range)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

# Create sequences of 60 days data to predict the next day's price
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Reshape for LSTM input: (samples, time steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print("X shape:", X.shape)
print("y shape:", y.shape)

# -----------------------------
# 5. Build the LSTM Model
# -----------------------------
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# -----------------------------
# 6. Train the Model
# -----------------------------
model.fit(X, y, batch_size=32, epochs=20)

# -----------------------------
# 7. Make Predictions
# -----------------------------
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)

# -----------------------------
# 8. Plot Predictions vs Actual
# -----------------------------
plt.figure(figsize=(10,6))
plt.plot(df.index[60:], data['Close'][60:], label='Actual Price')
plt.plot(df.index[60:], predictions, label='Predicted Price')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.legend()
plt.show()


^C


ModuleNotFoundError: No module named 'tensorflow'